In [26]:
import re

In [1]:
from opentrons.simulate import simulate, format_runlog

/Users/anton/.opentrons/deck_calibration.json not found. Loading defaults
/Users/anton/.opentrons/robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


In [64]:
# read the file
# protocol_file = open('../protocols/station-C-qpcr-map.py')
protocol_file = open('../experiments/20200319-end-to-end-qpcr-1/station-C-qpcr-map.py')
# simulate() the protocol, keeping the runlog
runlog, _bundle = simulate(protocol_file)

['Endogenous', 'Reaction', 'Reaction', 'Endogenous', 'Reaction', 'Reaction', '', 'Endogenous', '', '', '', 'Standard', 'Endogenous', 'Reaction', 'Reaction', 'Endogenous', 'Reaction', 'Reaction', '', 'Endogenous', '', '', '', 'Standard', 'Endogenous', 'Reaction', 'Reaction', 'Endogenous', 'Reaction', 'Reaction', '', 'Reaction', '', '', '', 'Standard', 'Endogenous', 'Reaction', 'Reaction', 'Endogenous', 'Reaction', 'Reaction', '', 'Reaction', '', '', '', 'Standard', 'Endogenous', 'Reaction', 'Reaction', 'Endogenous', 'Reaction', 'Reaction', '', '', '', '', '', 'Standard', 'Endogenous', 'Reaction', 'Reaction', 'Endogenous', 'Reaction', 'Reaction', '', '', '', '', '', 'Standard', 'Endogenous', 'Reaction', 'Reaction', 'Endogenous', 'Reaction', 'Reaction', '', 'Standard', '', '', '', 'Standard', 'Endogenous', 'Reaction', 'Reaction', 'Endogenous', 'Reaction', 'Reaction', '', 'Reaction', '', '', '', 'Standard']


In [58]:
# print the runlog
print(format_runlog(runlog))

Setting Temperature Module temperature to 4.0 °C (rounded off to nearest integer)
Picking up tip from A1 of Opentrons 96 Filter Tip Rack 20 µL on 2
Transferring 15.0 from A1 of Opentrons 24 Tube Rack with NEST 1.5 mL Snapcap on 5 to A1 of Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module on 4
	Aspirating 15.0 uL from A1 of Opentrons 24 Tube Rack with NEST 1.5 mL Snapcap on 5 at 1.0 speed
	Dispensing 15.0 uL into A1 of Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module on 4 at 1.0 speed
Blowing out at A1 of Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module on 4
Transferring 15.0 from A1 of Opentrons 24 Tube Rack with NEST 1.5 mL Snapcap on 5 to A6 of Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module on 4
	Aspirating 15.0 uL from A1 of Opentrons 24 Tube Rack with NEST 1.5 mL Snapcap on 5 at 1.0 speed
	Dispensing 15.0 uL into A6 of Opentrons 96 Well Aluminum B

In [65]:
gottip = False
source_wells = set()
dest_plates = dict()

for action in runlog:
    text = action['payload']['text'].strip()
    if text.startswith("Picking up tip"):
        gottip = True
    
    if text.startswith("Aspirating"):
        well, plate = re.match(r'Aspirating .* from ([A-Z][0-9]+) of (.* on .*) at .*', text).groups()
        source_wells.add((plate, well))
        
    if text.startswith("Dispensing"):
        dest_well, dest_plate = re.match(r'Dispensing .* into ([A-Z][0-9]+) of (.* on .*) at .*', text).groups()
        if dest_plate not in dest_plates:
            dest_plates[dest_plate] = dict()
            
        if dest_well not in dest_plates[dest_plate]:
            dest_plates[dest_plate][dest_well] = set()
        
        for source_well in source_wells:
            dest_plates[dest_plate][dest_well].add(source_well)
        
    if text.startswith("Dropping tip"):
        gottip = False
        source_wells.clear()
        

In [66]:
REAGENT_LOCATIONS = {
    'Endogenous': 'A1',
    'Reaction': 'A2',
    'Standard': 'A3',
    'Water': 'A4',
    'Control RNA': 'C1',
    'Control RNA 2': 'C2',
    'PCD 8': 'C5',
    'PCD 7': 'C6',
    'PCD 6': 'D1',
    'PCD 5': 'D2',
    'PCD 4': 'D3',
    'PCD 3': 'D4',
    'PCD 2': 'D5',
    'PCD 1': 'D6'
}

reagent_names = dict()
for name, loc in REAGENT_LOCATIONS.items():
    reagent_names[loc] = name

for name, plate in dest_plates.items():
    print(name)
    for well_name, well in sorted(plate.items(), key=lambda item: (int(item[0][1:]), item[0][0])):
        print(well_name)
        for source_plate, source_well in well:
            if source_plate == 'Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module on 4':
                continue
                
            if source_plate == 'Opentrons 24 Tube Rack with NEST 1.5 mL Snapcap on 5':
                print("  " + reagent_names[source_well])
            else:
                print("  " + source_well)
        print()
    print()
    print()

Opentrons 96 Well Aluminum Block with NEST Well Plate 100 µL on Temperature Module on 4
A1
  Endogenous
  A1

B1
  Endogenous
  B1

C1
  Endogenous
  C1

D1
  Endogenous
  D1

E1
  Endogenous
  E1

F1
  Endogenous
  F1

G1
  Endogenous
  G1

H1
  Endogenous
  H1

A2
  Reaction
  A1

B2
  Reaction
  B1

C2
  Reaction
  C1

D2
  D1
  Reaction

E2
  E1
  Reaction

F2
  F1
  Reaction

G2
  Reaction
  G1

H2
  H1
  Reaction

A3
  Reaction
  A1

B3
  Reaction
  B1

C3
  Reaction
  C1

D3
  D1
  Reaction

E3
  E1
  Reaction

F3
  F1
  Reaction

G3
  Reaction
  G1

H3
  H1
  Reaction

A4
  A2
  Endogenous

B4
  Endogenous
  B2

C4
  Endogenous
  C2

D4
  D2
  Endogenous

E4
  Endogenous
  E2

F4
  Endogenous
  F2

G4
  G2
  Endogenous

H4
  Endogenous
  H2

A5
  A2
  Reaction

B5
  B2
  Reaction

C5
  Reaction
  C2

D5
  D2
  Reaction

E5
  Reaction
  E2

F5
  Reaction
  F2

G5
  G2
  Reaction

H5
  Reaction
  H2

A6
  A2
  Reaction

B6
  B2
  Reaction

C6
  Reaction
  C2

D6
  D2
  Reaction

